# PolarPlot

Notebook to evaluate experiments of polar response of MEMS microphones.

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [ ]:
folder = '../data/polar_data'
angles = np.arange(5) * 45
els, azs = np.meshgrid(angles, angles)

data_full = pd.DataFrame()

for el, az in zip(els.flatten(), azs.flatten()):
    fname = f'{folder}/Mes_b_{el}_{az}.txt'
    try:
        data = pd.read_csv(fname, sep=' ', comment='*', names=['freq', 'SPL', 'phase'])
        data['elevation'] = el
        data['azimuth'] = az
        data_full = data_full.append(data)
    except Exception as e:
        print(e)

In [ ]:
fig, ax = plt.subplots()
for el, data_el in data_full.groupby('elevation'):
    for az, data in data_el.groupby('azimuth'):
        #ax.semilogy(data.freq[200:], data.SPL[200:], label=f'{el}/{az}')
        ax.loglog(data.freq[200:], data.SPL[200:], label=f'{el}/{az}')
        
ax.set_ylim(50, 100)
ax.legend(bbox_to_anchor=[1.0, 0.0], loc='lower left')
ax.set_xlabel('frequency [Hz]')
ax.set_ylabel('SPL')
data_full.head()

In [ ]:
# elevation
el = 45
octaves = 40*2**np.arange(10)
octaves = np.r_[0, octaves]
#octaves = np.linspace(data.freq.min(), data.freq.max(), 10)
print(octaves)

#ax = plt.subplot(111, projection='polar')
ax = plt.subplot(111)
for i in range(len(octaves)-1):
    mean_per_angle = []
    for az, data in data_full.groupby('azimuth'):
        o_min = octaves[i]
        o_max = octaves[i+1]
        this_data = data[(data.freq < o_max) & (data.freq > o_min)]
        mean = this_data.SPL.mean()
        if not this_data.empty:
            mean_per_angle.append(mean)
        else:
            mean_per_angle.append(None)
            
    ax.semilogy(angles*np.pi/180, mean_per_angle, label=f'{o_min:.0f}-{o_max:.0f}', color=f'C{i}',
            marker='*')
ax.legend(bbox_to_anchor=[1.0, 0.0], loc='lower left', title='frequency ranges')